<a href="https://www.kaggle.com/code/ibrahimawad02/titanic-competition?scriptVersionId=145020513" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.preprocessing import StandardScaler

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
dftr = pd.read_csv('/kaggle/input/titanic/train.csv', sep = ',', encoding = 'utf-8')
dftst = pd.read_csv('/kaggle/input/titanic/test.csv', sep = ',', encoding = 'utf-8')
dftr

In [ ]:
dftst

In [ ]:
dftr.isnull().sum()/dftr.shape[0]*100

In [ ]:
dftst.isnull().sum()/dftr.shape[0]*100

## Filling Null values

In [ ]:
pip install miceforest

In [ ]:
dftr.drop('Cabin', axis = 1, inplace = True)
dftst.drop('Cabin', axis = 1, inplace = True)

In [ ]:
#Changing the object columns into category type
dftr[dftr.select_dtypes(['object']).columns] = dftr.select_dtypes(['object']).apply(lambda x: x.astype('category'))

In [ ]:
dftr.info()

In [ ]:
from miceforest import ImputationKernel

mice_kernel = ImputationKernel(
data = dftr,
save_all_iterations = True,
random_state = 2024
)

In [ ]:
mice_kernel.mice(2)
dftr = mice_kernel.complete_data()
dftr.isnull().sum()

In [ ]:
#Changing the object columns into category type
dftst[dftst.select_dtypes(['object']).columns] = dftst.select_dtypes(['object']).apply(lambda x: x.astype('category'))

In [ ]:
dftst.info()

In [ ]:
mice_kernel = ImputationKernel(
data = dftst,
save_all_iterations = True,
random_state = 2024
)

In [ ]:
mice_kernel.mice(2)
dftst = mice_kernel.complete_data()
dftst.isnull().sum()

## Data preprocessing

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
dftr.describe(include = 'category')

In [ ]:
label_encoder = LabelEncoder()
category_columns = ['Sex', 'Ticket', 'Embarked']

for col in category_columns:
    dftr[col] = label_encoder.fit_transform(dftr[col])

# After Encoding
dftr.info()

In [ ]:
for col in category_columns:
    dftst[col] = label_encoder.fit_transform(dftst[col])

# After Encoding
dftst.info()

# Modelling
## Machine Learning

### Random Forest Classifier

In [ ]:
# Drop name and passenger ID columns since it will not affect the model
x_train = dftr.drop(['Name', 'PassengerId', 'Survived'], axis = 1)
y_train = dftr['Survived']
x_test = dftst.drop(['Name', 'PassengerId'], axis = 1)

In [ ]:
rf = RandomForestClassifier()

In [ ]:
rf.fit(x_train, y_train)

In [ ]:
rf.score(x_train, y_train)

In [ ]:
y_pred_rf = rf.predict(x_test)

In [ ]:
RF = pd.DataFrame({'PassengerID': dftst['PassengerId'],'Survived' : y_pred_rf})

In [ ]:
RF

In [ ]:
RF.to_csv('RF.csv', index=False)

### AdaBoost

In [ ]:
ada = AdaBoostClassifier()

In [ ]:
ada.fit(x_train, y_train)

In [ ]:
ada.score(x_train, y_train)

In [ ]:
y_pred_ada = ada.predict(x_test)

In [ ]:
ADA_df = pd.DataFrame({'PassengerID': dftst['PassengerId'],'Survived' : y_pred_ada})

In [ ]:
ADA_df

In [ ]:
ADA_df.to_csv('ADA.csv', index=False)

# Modelling : 
## Deep Learning

In [ ]:
X_train_scaled = StandardScaler().fit_transform(x_train)
X_test_scaled = StandardScaler().fit_transform(x_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization
from tensorflow.keras.regularizers import L1,L2
from tensorflow.keras.activations import relu,tanh,sigmoid

In [ ]:
model=Sequential()

#Input layer

model.add(Dense(8,activation='relu',kernel_initializer='he_uniform',kernel_regularizer=L2(0.01)))

#Hidden_layer

model.add(Dense(300,activation='relu',kernel_initializer='he_uniform',kernel_regularizer=L2(0.01)))

model.add(Dense(300,activation='relu',kernel_initializer='he_uniform',kernel_regularizer=L2(0.01)))

model.add(Dense(200,activation='relu',kernel_initializer='he_uniform',kernel_regularizer=L2(0.01)))

model.add(Dense(200,activation='relu',kernel_initializer='he_uniform',kernel_regularizer=L2(0.01)))

model.add(Dense(175,activation='relu',kernel_initializer='he_uniform',kernel_regularizer=L2(0.01)))

model.add(Dense(175,activation='relu',kernel_initializer='he_uniform',kernel_regularizer=L2(0.01)))

model.add(Dense(90,activation='relu',kernel_initializer='he_uniform',kernel_regularizer=L2(0.01)))

model.add(Dense(50,activation='relu',kernel_initializer='he_uniform',kernel_regularizer=L2(0.01)))

model.add(Dense(40,activation='relu',kernel_initializer='he_uniform',kernel_regularizer=L2(0.01)))

model.add(Dense(30,activation='relu',kernel_initializer='he_uniform',kernel_regularizer=L2(0.01)))

model.add(Dense(25,activation='relu',kernel_initializer='he_uniform',kernel_regularizer=L2(0.01)))

model.add(Dense(20,activation='relu',kernel_initializer='he_uniform',kernel_regularizer=L2(0.01)))

model.add(Dense(10,activation='relu',kernel_initializer='he_uniform',kernel_regularizer=L2(0.01)))

model.add(Dense(10,activation='relu',kernel_initializer='he_uniform',kernel_regularizer=L2(0.01)))


#Output Layer

model.add(Dense(1,activation='sigmoid'))




In [ ]:
model.compile(optimizer='SGD',loss='binary_crossentropy',metrics='accuracy')

In [ ]:
model.fit(X_train_scaled,y_train,epochs=300)

In [ ]:
model.summary()

In [ ]:
Model_trial_results=pd.DataFrame(model.history.history)

In [ ]:
Model_trial_results.head()

In [ ]:
Model_trial_results[['loss']].plot()

In [ ]:
Model_trial_results[['accuracy']].plot()

In [ ]:
model.evaluate(X_train_scaled,y_train)

In [ ]:
y_dl = model.predict(x_test)

In [ ]:
y_dl

In [ ]:
y_dl = (y_dl >= 0.5).astype(int)

In [ ]:
y_dl

In [ ]:
y_dl = y_dl.flatten()

In [ ]:
DL_df = pd.DataFrame({'PassengerID': dftst['PassengerId'],'Survived' : y_dl})

In [ ]:
DL_df

In [ ]:
DL_df.to_csv('DL.csv', index=False)

## Model 2

In [ ]:
model2=Sequential()

#Input layer

model2.add(Dense(8,activation='elu',kernel_initializer='he_uniform'))

#Hidden_layer

model2.add(Dense(1000,activation='elu',kernel_initializer='he_uniform'))

model2.add(BatchNormalization())
model2.add(Dropout(0.5))

model2.add(Dense(1000,activation='elu',kernel_initializer='he_uniform'))

model2.add(BatchNormalization())
model2.add(Dropout(0.5))

model2.add(Dense(700,activation='elu',kernel_initializer='he_uniform'))

model2.add(BatchNormalization())
model2.add(Dropout(0.5))

model2.add(Dense(650,activation='elu',kernel_initializer='he_uniform'))

model2.add(BatchNormalization())
model2.add(Dropout(0.5))

model2.add(Dense(500,activation='elu',kernel_initializer='he_uniform'))

model2.add(Dense(400,activation='elu',kernel_initializer='he_uniform'))

model2.add(BatchNormalization())
model2.add(Dropout(0.5))

model2.add(Dense(300,activation='elu',kernel_initializer='he_uniform'))


model2.add(Dense(300,activation='elu',kernel_initializer='he_uniform'))

model2.add(BatchNormalization())
model2.add(Dropout(0.5))

model2.add(Dense(200,activation='elu',kernel_initializer='he_uniform'))

model2.add(BatchNormalization())
model2.add(Dropout(0.5))

model2.add(Dense(200,activation='elu',kernel_initializer='he_uniform'))


model2.add(Dense(100,activation='elu',kernel_initializer='he_uniform'))

model2.add(BatchNormalization())
model2.add(Dropout(0.5))

model2.add(Dense(100,activation='elu',kernel_initializer='he_uniform'))

model2.add(Dense(50,activation='elu',kernel_initializer='he_uniform'))

model2.add(Dense(50,activation='elu',kernel_initializer='he_uniform'))

model2.add(BatchNormalization())
model2.add(Dropout(0.5))

model2.add(Dense(10,activation='elu',kernel_initializer='he_uniform'))


model2.add(Dense(10,activation='elu',kernel_initializer='he_uniform'))


model2.add(BatchNormalization())
model2.add(Dropout(0.5))


#Output Layer

model2.add(Dense(1,activation='sigmoid'))

In [ ]:
model2.compile(optimizer='Adam',loss='binary_crossentropy',metrics='accuracy')

model2.fit(X_train_scaled,y_train,epochs=150)

In [ ]:
Model2_trial_results=pd.DataFrame(model2.history.history)

Model2_trial_results.head()

plt.figure(figsize=(12,8))
Model2_trial_results[['loss']].plot()

Model2_trial_results[['accuracy']].plot()

In [ ]:
model2.evaluate(X_train_scaled,y_train)

In [ ]:
y_dl2 = model2.predict(X_test_scaled)

In [ ]:
y_dl2 = (y_dl >= 0.5).astype(int)

In [ ]:
y_dl2 = y_dl2.flatten()

In [ ]:
DL_df = pd.DataFrame({'PassengerID': dftst['PassengerId'],'Survived' : y_dl2})

In [ ]:
DL_df

In [ ]:
DL_df.to_csv('DL2.csv', index=False)

# Thanks for reading